In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

## Los Angeles County

In [ ]:
la_hispanic_2021 = pd.read_csv('https://raw.githubusercontent.com/jyyang7/reverse_mortgage/main/all_mortgage_data/la_hispanic_2021.csv', dtype={
    'county_code': str,
    'census_tract': str
})

In [ ]:
rm_la_hispanic_2021 = la_hispanic_2021[(la_hispanic_2021['reverse_mortgage'] == 1)]
rm_la_hispanic_2021

In [ ]:
rm_la_hispanic_2021.to_csv('./reverse_mortgage_data/rm_la_hispanic_2021.csv', index=False)

In [ ]:
#read csv

la_all_2018 = pd.read_csv('./all_mortgage_data/la_all_2018.csv', dtype={
    'county_code': str,
    'census_tract': str
})

la_all_2019 = pd.read_csv('./all_mortgage_data/la_all_2019.csv', dtype={
    'county_code': str,
    'census_tract': str
})

la_all_2020 = pd.read_csv('./all_mortgage_data/la_all_2020.csv', dtype={
    'county_code': str,
    'census_tract': str
})

la_all_2021 = pd.read_csv('./all_mortgage_data/la_all_2021.csv', dtype={
    'county_code': str,
    'census_tract': str
})

In [ ]:
# filter to reverse mortgage

rm_la_all_2018 = la_all_2018[(la_all_2018['reverse_mortgage'] == 1)]
rm_la_all_2019 = la_all_2019[(la_all_2019['reverse_mortgage'] == 1)]
rm_la_all_2020 = la_all_2020[(la_all_2020['reverse_mortgage'] == 1)]
rm_la_all_2021 = la_all_2021[(la_all_2021['reverse_mortgage'] == 1)]

In [ ]:
rm_la_all_2018.info()

In [ ]:
rm_la_all_2019.info()

In [ ]:
rm_la_all_2020.info()

In [ ]:
rm_la_all_2021.info()

In [ ]:
# concatenate 2018-2021 datasets

rm_la_all = pd.concat((rm_la_all_2018, rm_la_all_2019, rm_la_all_2020, rm_la_all_2021), axis=0)

In [ ]:
rm_la_all.sample(10)

In [ ]:
# export to csv

rm_la_all.to_csv('./reverse_mortgage_data/rm_la_all.csv', index=False)

## Alameda County

In [ ]:
# read csv

alameda_all_2018 = pd.read_csv('./all_mortgage_data/sac_all_2018.csv', dtype={
    'county_code': str,
    'census_tract': str
})

sac_all_2019 = pd.read_csv('./all_mortgage_data/alameda_all_2019.csv', dtype={
    'county_code': str,
    'census_tract': str
})

alameda_all_2020 = pd.read_csv('./all_mortgage_data/alameda_all_2020.csv', dtype={
    'county_code': str,
    'census_tract': str
})

alameda_all_2021 = pd.read_csv('./all_mortgage_data/alameda_all_2021.csv', dtype={
    'county_code': str,
    'census_tract': str
})

In [ ]:
# filter to reverse mortgage

alameda_rm_2018 = alameda_all_2018[(alameda_all_2018['reverse_mortgage'] == 1)]
alameda_rm_2019 = alameda_all_2019[(alameda_all_2019['reverse_mortgage'] == 1)]
alameda_rm_2020 = alameda_all_2020[(alameda_all_2020['reverse_mortgage'] == 1)]
alameda_rm_2021 = alameda_all_2021[(alameda_all_2021['reverse_mortgage'] == 1)]

In [ ]:
# concatenate 2018-2021 years

alameda_rm_all = pd.concat((alameda_rm_2018, alameda_rm_2019, alameda_rm_2020, alameda_rm_2021), axis=0)
alameda_rm_all.info()

In [ ]:
# export
alameda_rm_all.to_csv('./reverse_mortgage_data/alameda_rm_all.csv', index=False)

## Sacramento County

In [ ]:
# read csv

sac_all_2018 = pd.read_csv('./all_mortgage_data/sac_all_2018.csv', dtype={
    'county_code': str,
    'census_tract': str
})

sac_all_2019 = pd.read_csv('./all_mortgage_data/sac_all_2019.csv', dtype={
    'county_code': str,
    'census_tract': str
})

sac_all_2020 = pd.read_csv('./all_mortgage_data/sac_all_2020.csv', dtype={
    'county_code': str,
    'census_tract': str
})

sac_all_2021 = pd.read_csv('./all_mortgage_data/sac_all_2021.csv', dtype={
    'county_code': str,
    'census_tract': str
})

In [ ]:
# filter to reverse mortgage

sac_rm_2018 = sac_all_2018[(sac_all_2018['reverse_mortgage'] == 1)]
sac_rm_2019 = sac_all_2019[(sac_all_2019['reverse_mortgage'] == 1)]
sac_rm_2020 = sac_all_2020[(sac_all_2020['reverse_mortgage'] == 1)]
sac_rm_2021 = sac_all_2021[(sac_all_2021['reverse_mortgage'] == 1)]

In [ ]:
# concatenate 2018-2021 years

sac_rm_all = pd.concat((sac_rm_2018, sac_rm_2019, sac_rm_2020, sac_rm_2021), axis=0)
sac_rm_all.info()

In [ ]:
# export
sac_rm_all.to_csv('./reverse_mortgage_data/sac_rm_all.csv', index=False)

## San Bernardino County